In [5]:
from preproc_tabular import get_tabular
from dataset_sales import getDataset
import sales_prediction as sp
from torch.optim.adamw import AdamW
from torch import nn
import torch
import os

img_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\ResizedImages\\'
desc_path= 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\descrizioni\\descrizioni.pt'
desc_tot_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\descrizioni\\descrizioni_tot.pt'

data_week = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\front_img_week.csv'
data_month = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\front_img_month.csv'
data_season = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\front_img_season.csv'

noneg_week = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\nonegozio_week.csv'
noneg_month = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\nonegozio_month.csv'
noneg_season = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\nonegozio_season.csv'

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Sto usando {device}")

modello = sp.create_model(0, 1).to(device)
batch_size = 128
epochs = 15
criterion = nn.MSELoss()

optimizer = AdamW(
    [
        {"params": modello.image.parameters(), "lr": 1e-3},
        {"params": modello.tabular.parameters()},
        {"params": modello.description.parameters(), "lr": 1e-3},
        {"params": modello.final.parameters()}
    ],
    lr=1e-2,
)
early_stop = sp.EarlyStopping(5, 0.001)

data, references, descriptions, target = get_tabular(data_week, desc_path)

train, val = getDataset(references, data, descriptions, target, img_path, batch_size, 0.28)

mse_final, mae_final, r2_final = 0, 0, 0
modello_final = 0
min_mae = 100000
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    sp.train_loop(train, modello, criterion, optimizer, batch_size, device)
    mse, mae, r2, bias = sp.validation_loop(val, modello, criterion, device)
    if (min_mae > mae):
        modello_final = modello
        min_mae = mae
    early_stp = early_stop(mse)
    
    if early_stp:
        print('Early Stop attivato.')
        break
    
    
print("Done!")

folder = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\results\\week_negozi'
if not os.path.isdir(folder):
    os.makedirs(folder)
model_sc = torch.jit.script(modello_final)
model_sc.save(f'{folder}\\weights.pt')

Sto usando cuda


c:\Users\GRVRLD00P\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
c:\Users\GRVRLD00P\Documents\Progetto ORS\Progetto ORS\ProgettoORS\sales_prediction\preproc_tabular.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted b

Epoch 1
-------------------------------
MSE: 14.389523  [   64/121327]
MSE: 4.867925  [ 6464/121327]
MSE: 6.911852  [12864/121327]
MSE: 8.736591  [19264/121327]
MSE: 9.492218  [25664/121327]
MSE: 26.807144  [32064/121327]
MSE: 6.211313  [38464/121327]
MSE: 5.936092  [44864/121327]
MSE: 9.306314  [51264/121327]
MSE: 7.519350  [57664/121327]
MSE: 5.679523  [64064/121327]
MSE: 10.397071  [70464/121327]
MSE: 4.391048  [76864/121327]
MSE: 6.418861  [83264/121327]
MSE: 5.629539  [89664/121327]
MSE: 5.127367  [96064/121327]
MSE: 14.754414  [102464/121327]
MSE: 7.644356  [108864/121327]
MSE: 6.720218  [115264/121327]
In questa epoca, l'MSE medio è 7.6495842933654785
Validation Error: 
 Avg MSE: 7.579702 
 Avg MAE: 1.821988 
 R2: 0.062378
 Bias: -0.403536

Epoch 2
-------------------------------
MSE: 5.061213  [   64/121327]
MSE: 4.360721  [ 6464/121327]
MSE: 6.514523  [12864/121327]
MSE: 7.426700  [19264/121327]
MSE: 8.714802  [25664/121327]
MSE: 24.569653  [32064/121327]
MSE: 6.080677  [38464

KeyboardInterrupt: 